<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/notebooks/%E3%80%90%E5%AE%9F%E9%A8%93%E7%89%88%E3%80%91%E5%8B%95%E7%94%BBURL%E3%81%8B%E3%82%89%E9%AB%98%E7%B2%BE%E5%BA%A6%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%E5%AE%9F%E8%A1%8C%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%83%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### **【完成版】動画URLから高精度文字起こし＆Gemini要約 実行スクリプト**

#以下の3つのセルを上から順番に実行してください。

#### **セル1: 環境構築**

# -------------------------------------------------------------------------------------------
# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# 必要なライブラリとツールをインストール
# yt-dlp: 様々なサイトから動画や音声をダウンロードするツール
# faster-whisper: 高速化されたWhisperモデル
# google-generativeai: Gemini APIを使用するためのライブラリ
# ffmpeg: 動画・音声の処理・変換に必須のツール
print("\n▼ 必要なライブラリとツールのインストール")
!pip install -U yt-dlp -q
!pip install faster-whisper==1.0.3 -q
!pip install google-genai -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q
!apt-get update && apt-get install -y ffmpeg -qq

print("\n✅ 環境構築が完了しました。")
print("次のセルに進んでGoogle Driveに接続してください。")

In [ ]:
#### **セル2: Google Driveへの接続 (任意)**
#文字起こししたテキストファイルをGoogle Driveに保存したい場合に、このセルを実行してGoogle Driveをマウント（接続）してください。

# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")
print("最後のセルに進んで、文字起こしを実行してください。")

In [ ]:
#### **セル3: 🚀 URLから高精度文字起こし＆要約実行**

#@title 🚀 URLから高精度文字起こし＆要約実行 (Gemini 2.5対応)
#@markdown ### 1. 動画のURLと出力先の設定
#@markdown ---
#@markdown 文字起こししたい動画のURLを入力してください。
video_url = "https://youtu.be/Q_b5C5neo7U?si=bJBuGKKHJzGIkYSs" #@param {type:"string"}

#@markdown 文字起こし結果（.txtファイル）を保存するフォルダのパスを指定します。
output_transcript_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `Zoont/...-int8-ct2`は精度を維持しつつ最速・省メモリな**総合推奨モデル**です。
model_name = "Zoont/faster-whisper-large-v3-turbo-int8-ct2" #@param ["Zoont/faster-whisper-large-v3-turbo-int8-ct2", "deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium", "small", "base", "tiny"]
#@markdown **計算タイプ**: `int8`モデルには`int8_float16`、`float16`モデルには`float16`の組み合わせを推奨します。
compute_type = "int8_float16" #@param ["int8_float16", "float16", "int8", "float32"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 200 #@param {type:"slider", min:100, max:2000, step:100}

#@markdown ### 4. 高度な設定（オプション）
#@markdown ---
#@markdown **ビームサイズ (beam_size)**: 文字起こしの精度と速度のトレードオフを調整します。
beam_size = 5 #@param {type:"slider", min:1, max:10, step:1}
#@markdown **中間ファイルのクリーンアップ**: チェックを入れると、処理終了後にダウンロードした音声ファイルを自動で削除します。
cleanup_audio_file = True #@param {type:"boolean"}

#@markdown ### 5. Geminiによる要約設定（オプション）
#@markdown ---
#@markdown 文字起こし結果の要約を有効にする場合は、チェックを入れてAPIキー等を設定してください。
enable_summarization = True #@param {type:"boolean"}
gemini_api_key = "" #@param {type:"string"}
gemini_model = "gemini-2.5-flash" #@param ["gemini-2.5-pro", "gemini-2.5-flash", "gemini-2.5-flash-lite"]
output_summary_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_summary" #@param {type:"string"}


# --- ここから下は実行コード（編集不要） ---
import os
import yt_dlp
from faster_whisper import WhisperModel
import time
from datetime import datetime, timezone, timedelta
import google.generativeai as genai

def get_current_timestamp():
    """現在時刻を[YYYY-MM-DD HH:MM:SS]形式の文字列で返す"""
    JST = timezone(timedelta(hours=+9))
    return datetime.now(JST).strftime('%Y-%m-%d %H:%M:%S')

def run_pipeline():
    """メインの処理パイプライン"""
    if not video_url:
        print("❌ エラー: 動画のURLが入力されていません。")
        return

    print(f"{get_current_timestamp()} --- 1. パイプライン開始 ---")

    # 出力ディレクトリの作成
    os.makedirs(output_transcript_dir, exist_ok=True)
    print(f"文字起こし出力先: {output_transcript_dir}")
    if enable_summarization:
        os.makedirs(output_summary_dir, exist_ok=True)
        print(f"要約出力先: {output_summary_dir}")


    # 一時ダウンロード用ディレクトリの作成
    temp_audio_dir = "/content/temp_audio"
    os.makedirs(temp_audio_dir, exist_ok=True)

    # 2. モデルのロード
    print(f"\n{get_current_timestamp()} --- 2. Whisperモデル '{model_name}' ({compute_type}) をロード中... ---")
    start_load_time = time.time()
    try:
        model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
    except Exception as e:
        print(f"❌ モデルのロード中にエラーが発生しました: {e}")
        print("   -> ランタイムのタイプがGPUになっているか確認してください。")
        return

    end_load_time = time.time()
    print(f"✅ Whisperモデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")

    # 3. yt-dlpによる音声のダウンロードと抽出
    print(f"\n{get_current_timestamp()} --- 3. URLから音声のダウンロードと抽出を開始 ---")
    print(f"対象URL: {video_url}")

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'wav', 'preferredquality': '192'}],
        'outtmpl': os.path.join(temp_audio_dir, '%(id)s.%(ext)s'),
        'quiet': True,
        'nocheckcertificate': True,
    }

    audio_file_path = None
    video_title = "untitled"
    video_id = "default_id"
    full_transcript_text = ""
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(video_url, download=True)
            video_id = info.get('id', 'default_id')
            video_title = info.get('title', 'untitled').replace('/', '／') # ファイル名に使えない文字を置換
            base, _ = os.path.splitext(ydl.prepare_filename(info))
            audio_file_path = base + ".wav"

        if not os.path.exists(audio_file_path):
            raise FileNotFoundError("音声ファイルの生成に失敗しました。")

        print(f"✅ 音声ファイルの準備が完了しました: {os.path.basename(audio_file_path)}")
        print(f"   動画タイトル: {video_title}")

    except Exception as e:
        print(f"❌ 音声のダウンロードまたは変換中にエラーが発生しました: {e}")
        return

    # 4. 文字起こし処理
    print(f"\n{get_current_timestamp()} --- 4. 文字起こし処理開始 ---")
    start_transcribe_time = time.time()
    try:
        print(f"  - 設定 (beam_size: {beam_size}, VAD: {'有効' if use_vad_filter else '無効'})")
        segments, info = model.transcribe(
            audio_file_path,
            beam_size=beam_size,
            vad_filter=use_vad_filter,
            vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
        )
        print(f"  - 検出言語: {info.language} (確率: {info.language_probability:.2f})")

        # 5. 結果の保存
        # ファイル名に使えない文字を置換し、長さを制限
        safe_title = "".join(c if c.isalnum() or c in (' ', '-', '_') else '_' for c in video_title).strip()
        output_txt_filename = f"{video_id}_{safe_title[:50]}.txt"
        transcript_output_path = os.path.join(output_transcript_dir, output_txt_filename)

        transcript_lines = []
        with open(transcript_output_path, "w", encoding="utf-8") as f:
            f.write(f"動画タイトル: {video_title}\n")
            f.write(f"URL: {video_url}\n")
            f.write(f"文字起こし実行日時: {get_current_timestamp()}\n")
            f.write(f"モデル: {model_name} ({compute_type})\n\n---\n\n")

            for segment in segments:
                line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
                f.write(line)
                transcript_lines.append(segment.text.strip())
            full_transcript_text = "\n".join(transcript_lines) # 要約用に全文を結合


        end_transcribe_time = time.time()
        print(f"✅ 文字起こしが完了しました。({end_transcribe_time - start_transcribe_time:.2f}秒)")
        print(f"   -> 結果を保存しました: {transcript_output_path}")

        # --- ここから要約処理 ---
        if enable_summarization:
            print(f"\n{get_current_timestamp()} --- 5. Geminiによる要約を開始 ---")
            if not gemini_api_key:
                print("  - ⚠️ Gemini APIキーが設定されていません。要約をスキップします。")
            elif not full_transcript_text.strip():
                print("  - ⚠️ 文字起こし結果が空のため、要約をスキップします。")
            else:
                try:
                    genai.configure(api_key=gemini_api_key)
                    model_gemini = genai.GenerativeModel(gemini_model)
                    prompt = f"以下の動画書き起こしテキストを、重要なポイントと動画の構成を含めて要約して最大コンテクストで出力してください。\n\n---\n\n{full_transcript_text}"
                    response = model_gemini.generate_content(prompt)
                    summary_text = response.text

                    # 要約結果をファイルに保存
                    summary_filename = f"{video_id}_{safe_title[:50]}_summary.txt"
                    summary_output_path = os.path.join(output_summary_dir, summary_filename)

                    with open(summary_output_path, "w", encoding="utf-8") as f:
                        f.write(f"■ 元動画タイトル: {video_title}\n")
                        f.write(f"■ URL: {video_url}\n")
                        f.write(f"■ 要約実行日時: {get_current_timestamp()}\n")
                        f.write(f"■ 使用モデル: {gemini_model}\n\n---\n\n")
                        f.write(summary_text)

                    print(f"  - ✅ 要約結果を保存しました: {summary_output_path}")

                except Exception as e:
                    print(f"  - 💥 Gemini APIエラー: 要約中にエラーが発生しました。 {e}")
        # --- 要約処理ここまで ---


    except Exception as e:
        print(f"❌ 文字起こし中にエラーが発生しました: {e}")
    finally:
        # 6. 中間ファイルのクリーンアップ
        if cleanup_audio_file and audio_file_path and os.path.exists(audio_file_path):
            os.remove(audio_file_path)
            print(f"\n{get_current_timestamp()} --- 6. 中間音声ファイルを削除しました ---")

    print(f"\n{get_current_timestamp()} --- 🎉 全ての処理が完了しました ---")

# パイプライン実行
run_pipeline()